# RAG Chatbot — University Regulations (Sharif University)

This notebook runs the full RAG pipeline:
1. Install dependencies
2. Upload/load knowledge base chunks
3. Build embeddings + FAISS index
4. Retrieval
5. LLM generation with Qwen2.5-7B-Instruct (4-bit)
6. Interactive chat + Evaluation

**Runtime:** Use GPU (T4 or better) via Runtime > Change runtime type

## Step 1: Install Dependencies

In [ ]:
!pip install -q torch transformers sentence-transformers faiss-cpu bitsandbytes accelerate gradio tqdm

## Step 2: Upload Knowledge Base

Upload `sharif_rules_chunks.json` from your `data/` folder, or clone the repo.

In [ ]:
import os
import json

# Option A: Clone repo (uncomment if using git)
# !git clone https://github.com/YOUR_USERNAME/G44-RAG.git
# os.chdir('G44-RAG')

# Option B: Upload file manually
from google.colab import files
uploaded = files.upload()  # upload sharif_rules_chunks.json

CHUNKS_PATH = 'sharif_rules_chunks.json'

with open(CHUNKS_PATH, 'r', encoding='utf-8') as f:
    chunks = json.load(f)

print(f'Loaded {len(chunks)} chunks')
print(f'Sample chunk keys: {list(chunks[0].keys())}')
print(f'Sample content: {chunks[1]["content"][:200]}...')

## Step 3: Build Embeddings + FAISS Index

In [ ]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

EMBEDDING_MODEL_NAME = 'intfloat/multilingual-e5-base'

print(f'Loading embedding model: {EMBEDDING_MODEL_NAME}')
embed_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

def prepare_texts(chunks):
    texts = []
    for chunk in chunks:
        header = f"{chunk['rule_title']} — {chunk['section_title']}"
        text = f"passage: {header}\n{chunk['content']}"
        texts.append(text)
    return texts

texts = prepare_texts(chunks)
print(f'Prepared {len(texts)} texts for embedding')

print('Computing embeddings...')
embeddings = embed_model.encode(texts, batch_size=32, show_progress_bar=True)
embeddings = np.array(embeddings, dtype='float32')
print(f'Embeddings shape: {embeddings.shape}')

faiss.normalize_L2(embeddings)
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)
print(f'FAISS index built with {index.ntotal} vectors (dim={dim})')

## Step 4: Retrieval Function

In [ ]:
def retrieve(query, top_k=5):
    query_text = f'query: {query}'
    query_embedding = embed_model.encode([query_text], normalize_embeddings=True).astype('float32')
    scores, indices = index.search(query_embedding, top_k)

    results = []
    for rank, (idx, score) in enumerate(zip(indices[0], scores[0])):
        if idx < 0:
            continue
        chunk = chunks[idx].copy()
        chunk['score'] = float(score)
        chunk['rank'] = rank + 1
        results.append(chunk)
    return results


def format_context(results):
    parts = []
    for r in results:
        header = f"[{r['rule_title']} | {r['section_title']}]"
        parts.append(f"{header}\n{r['content']}")
    return '\n\n---\n\n'.join(parts)


# Test retrieval
test_query = 'شرایط مشروطی دانشجو چیست؟'
results = retrieve(test_query, top_k=3)
print(f'Query: {test_query}\n')
for r in results:
    print(f"[Rank {r['rank']}] Score: {r['score']:.4f}")
    print(f"  Rule: {r['rule_title']}")
    print(f"  Section: {r['section_title']}")
    print(f"  Content: {r['content'][:150]}...")
    print()

## Step 5: Load LLM (Qwen2.5-7B-Instruct, 4-bit)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = 'Qwen/Qwen2.5-7B-Instruct'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
)

print(f'Loading model: {MODEL_NAME} (4-bit quantized)...')
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map='auto',
    trust_remote_code=True,
)
model.eval()
print('Model loaded successfully!')
print(f'Device: {model.device}')

## Step 6: Generation (Prompt Engineering)

In [ ]:
SYSTEM_PROMPT = """تو یک دستیار هوشمند دانشگاهی هستی که فقط بر اساس آیین‌نامه‌ها و مقررات رسمی دانشگاه صنعتی شریف پاسخ می‌دهی.

قوانین پاسخ‌دهی:
۱. فقط بر اساس متن مقررات ارائه‌شده پاسخ بده. هرگز اطلاعاتی خارج از این متون اضافه نکن.
۲. در پاسخ، حتماً نام آیین‌نامه و در صورت امکان شماره ماده، بند یا تبصره را ذکر کن.
۳. پاسخ را کوتاه، دقیق و مستقیم بنویس.
۴. اگر پاسخ سوال در متون ارائه‌شده وجود ندارد، بگو: «اطلاعاتی در مورد این سوال در آیین‌نامه‌های موجود یافت نشد. لطفاً از اداره آموزش استعلام بگیرید.»
۵. هرگز قانون یا تفسیری از خودت اختراع نکن.
۶. به زبان فارسی و رسمی پاسخ بده."""


def generate_answer(query, context, max_new_tokens=512, temperature=0.3):
    user_message = f"""بر اساس متون آیین‌نامه‌ای زیر، به سوال دانشجو پاسخ بده.

--- متون مرتبط ---
{context}
--- پایان متون ---

سوال: {query}"""

    messages = [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': user_message},
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors='pt').to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.1,
        )

    new_tokens = output_ids[0][inputs['input_ids'].shape[1]:]
    return tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

## Step 7: Full RAG Pipeline

In [ ]:
def rag_answer(query, top_k=5):
    retrieved = retrieve(query, top_k=top_k)
    context = format_context(retrieved)
    answer = generate_answer(query, context)

    return {
        'query': query,
        'answer': answer,
        'sources': [
            {
                'rule_title': r['rule_title'],
                'section_title': r['section_title'],
                'score': r['score'],
                'content_preview': r['content'][:200],
            }
            for r in retrieved
        ],
    }


# Test the full pipeline
result = rag_answer('شرایط مشروطی دانشجوی کارشناسی چیست؟')
print(f"Q: {result['query']}")
print(f"\nA: {result['answer']}")
print(f"\nSources:")
for s in result['sources']:
    print(f"  - {s['rule_title']} | {s['section_title']} (score: {s['score']:.4f})")

## Step 8: Interactive Chat (Gradio)

In [ ]:
import gradio as gr


def chat_fn(query, history):
    if not query.strip():
        return 'لطفاً سوال خود را وارد کنید.'

    result = rag_answer(query, top_k=5)
    answer = result['answer']

    sources_text = '\n\n---\n**منابع:**\n'
    for s in result['sources']:
        sources_text += f"- {s['rule_title']} — {s['section_title']} (امتیاز: {s['score']:.3f})\n"

    return answer + sources_text


demo = gr.ChatInterface(
    fn=chat_fn,
    title='چت‌بات راهنمای مقررات آموزشی دانشگاه صنعتی شریف',
    description='سوالات خود درباره آیین‌نامه‌ها و مقررات آموزشی را بپرسید.',
    examples=[
        'شرایط مشروطی دانشجو چیست؟',
        'حداکثر سنوات مجاز تحصیل در مقطع کارشناسی چقدر است؟',
        'شرایط حذف اضطراری درس چیست؟',
        'آیا استفاده از هوش مصنوعی در تکالیف مجاز است؟',
        'قوانین کارآموزی چیست؟',
    ],
)

demo.launch(share=True, debug=True)

## Step 9: Evaluation on Question Set

In [ ]:
import csv
from datetime import datetime

EVAL_QUESTIONS = [
    'شرایط مشروطی دانشجوی کارشناسی چیست؟',
    'حداکثر سنوات مجاز تحصیل در دوره کارشناسی چقدر است؟',
    'آیا استفاده از ابزار هوش مصنوعی در تکالیف درسی مجاز است؟',
    'شرایط حذف اضطراری درس چیست؟',
    'قوانین غیبت در امتحان پایان‌ترم چیست؟',
    'شرایط معرفی به استاد چگونه است؟',
    'قوانین کارآموزی در دوره کارشناسی چیست؟',
    'شرایط مهمانی دانشجو در دانشگاه دیگر چگونه است؟',
    'قوانین تغییر رشته در دوره کارشناسی چیست؟',
    'شرایط پروژه کارشناسی چگونه است؟',
    'آیین‌نامه دوره کوآپ چه مقرراتی دارد؟',
    'شرایط انتقال به دانشگاه صنعتی شریف چیست؟',
    'مهلت فراغت از تحصیل چقدر است؟',
    'حداقل و حداکثر واحد مجاز در هر ترم چقدر است؟',
    'شرایط دستیاری آموزشی چیست؟',
]

print(f'Running evaluation on {len(EVAL_QUESTIONS)} questions...\n')

eval_results = []
for i, question in enumerate(EVAL_QUESTIONS, 1):
    print(f'[{i}/{len(EVAL_QUESTIONS)}] {question}')
    result = rag_answer(question, top_k=5)
    eval_results.append(result)
    print(f'  → {result["answer"][:120]}...')
    print()

# Save JSON
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
json_path = f'eval_results_{timestamp}.json'
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(eval_results, f, ensure_ascii=False, indent=2)
print(f'JSON saved: {json_path}')

# Save CSV
csv_path = f'eval_results_{timestamp}.csv'
with open(csv_path, 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['question', 'answer', 'num_sources', 'source_1_title', 'source_1_section', 'source_1_score'])
    for r in eval_results:
        top = r['sources'][0] if r['sources'] else {}
        writer.writerow([
            r['query'], r['answer'], len(r['sources']),
            top.get('rule_title', ''), top.get('section_title', ''),
            f"{top.get('score', 0):.4f}",
        ])
print(f'CSV saved: {csv_path}')

# Download files
from google.colab import files
files.download(json_path)
files.download(csv_path)